# Реализация модели защищенного канала передачи данных по спецификации TLS 1.3

In [1]:
IS_DEBUG = 1

In [2]:
def trace(*args, **kwargs):
    """
    Отладочная трассировка
    """
    
    global IS_DEBUG
    if IS_DEBUG:
        print('[TRACE]', end=' ')
        print(*args, **kwargs)

----

In [3]:
# Криптография из различных ГОСТ'ов
from pygost import mgm
from pygost import gost3412
from pygost import gost3410
from pygost import gost34112012256

# Криптография из западных стандартов
from Crypto.Cipher import AES
from Crypto.Hash import SHA256

# HKDF
from Crypto.Protocol.KDF import HKDF

# КГПСЧ
from Crypto.Random import get_random_bytes, random
from Crypto.Util.number import getStrongPrime

# Различные утилиты
from collections import namedtuple
from Crypto.Util import number
from functools import partial
from binascii import hexlify
from enum import Enum
import uuid
import gmpy2
import pickle

----

## Реализация оберток над криптографическими алгоритмами

In [4]:
def multiply_by_scalar(curve, scalar, point):
    """
    Умножение точки эллиптической кривой на скаляр 
    """
    
    return curve.exp(scalar, *point)


def add_points(curve, lhs, rhs):
    """
    Сложение двух точек эллиптической кривой
    """
    
    return curve._add(*lhs, *rhs)

---

In [5]:
#
# Сертификат это простая структурка, поэтому обойдусь именованным кортежем
#

Certificate = namedtuple('Certificate', ['id', 'public_key', 'signature'])

----

In [6]:
class KuznyechikMGM(object):
    """
    Класс-обертка над Кузнечиком в режиме MGM
    """
    
    KEY_SIZE = 32
    
    def __init__(self, key: bytes):
        """
        Инициализация - сохраняю ключ
        """

        self._key = key
    
    
    def encrypt(self, plaintext: bytes):
        """
        Зашифрование шифром Кузнечик в режиме MGM
        """
        
        encrypter = partial(KuznyechikMGM._encrypt_block, self._key)
        cipher = mgm.MGM(encrypter, KuznyechikMGM._block_size())
        
        nonce = KuznyechikMGM._generate_nonce()
        ct_with_tag = cipher.seal(nonce, plaintext, b'')
        
        return nonce, ct_with_tag[:-cipher.tag_size], ct_with_tag[-cipher.tag_size:]
    
    
    def decrypt(self, nonce: bytes, ciphertext: bytes, tag: bytes):
        """
        Расшифрование шифром Кузнечик в режиме MGM
        """
        
        encrypter = partial(KuznyechikMGM._encrypt_block, self._key)
        cipher = mgm.MGM(encrypter, KuznyechikMGM._block_size())
            
        ct_with_tag = ciphertext + tag
            
        return cipher.open(nonce, ct_with_tag, b'')
        
        
    @staticmethod
    def _block_size():
        return gost3412.GOST3412Kuznechik.blocksize
        
        
    @staticmethod
    def _encrypt_block(key: bytes, block: bytes):
        cipher = gost3412.GOST3412Kuznechik(key)
        return cipher.encrypt(block)
    
    
    @staticmethod
    def _generate_nonce():
        nonce = get_random_bytes(KuznyechikMGM._block_size())
        while bytearray(nonce)[0] & 0x80 > 0:
            nonce = get_random_bytes(KuznyechikMGM._block_size())
            
        return nonce

In [7]:
class AESGCM(object):
    """
    Класс-обертка над AES в режиме GCM
    """
    
    KEY_SIZE = 16
    
    def __init__(self, key: bytes):
        """
        Инициализация - сохраняю ключ
        """

        self._key = key
        
        
    def encrypt(self, plaintext: bytes):
        """
        Зашифрование шифром AES в режиме GCM
        """
        
        cipher = AES.new(self._key, AES.MODE_GCM)
        return cipher.nonce, *cipher.encrypt_and_digest(plaintext)
    
    
    def decrypt(self, nonce: bytes, ciphertext: bytes, tag: bytes):
        """
        Зашифрование шифром AES в режиме GCM
        """
        
        cipher = AES.new(self._key, AES.MODE_GCM, nonce=nonce)
        return cipher.decrypt_and_verify(ciphertext, tag)

----

In [8]:
class Streebog(gost34112012256.GOST34112012256):
    """
    pycryptodome-совместимая обертка над хэш-функцией Стрибог
    """
    
    digest_size = 32
    
    def __init__(self, data=None):
        """
        Инициализация и обновление состояния (если надо)
        """
        
        super(Streebog, self).__init__()
        if data is not None:
            self.update(data)
    
    @staticmethod
    def new(data=None):
        """
        Создание нового объекта
        """
        
        return Streebog(data)

In [9]:
def hash_digest(hasher_type, *args):
    """
    Вычисление хэша при помощи объекта-обертки
    """
    
    hasher = hasher_type.new()
    
    for arg in args:
        hasher.update(arg)
        
    return hasher.digest()

----

In [10]:
class DiffieHellmanProcessor(object):
    """
    Обработчик обмена ключами по протоколу Диффи-Хеллмана
    """
    
    def __init__(self, group=None, generator=None):
        """
        Инициализация - выбор группы, образующего элемента
        Возможно конструирование по готовой группе и образующему
        """

        self._group = Integers(getStrongPrime(512)) if group is None else group
        self._generator = random.randint(int(2), int(self._group.order() - 2)) if generator is None \
            else generator
        
        self._modulus = self._group.order()
        
        trace('[DiffieHellmanProcessor]', f'Using group of order {self._modulus}')
        
    
    def generate_private_key(self):
        """
        Генерирование закрытого ключа
        """
        
        return random.randint(int(2), int(self.order - 2))
    
    
    def get_public_key(self, private_key):
        """
        Получение открытого ключа по закрытому
        """
        
        return self.exponentiate(private_key, self._generator)
    
    
    def exponentiate(self, scalar, element):
        """
        Возведение в степень элемента группы
        """
        
        return self._group(int(gmpy2.powmod(int(element), int(scalar), self.order)))
    
    
    def element_to_bytes(self, element):
        """
        Конвертация элемента группы в байтовое представление
        """
        
        return number.long_to_bytes(int(element))
    
    
    @property
    def group(self):
        """
        Получение группы
        """
        
        return self._group
    
        
    @property
    def generator(self):
        """
        Получение образующего элемента
        """
        
        return self._generator
    
    
    @property
    def order(self):
        """
        Получение порядка группы
        """
        
        return self._modulus

In [11]:
class EllipticCurveDiffieHellmanProcessor(object):
    """
    Обработчик обмена ключами по протоколу Диффи-Хеллмана
    в группе точек эллиптической кривой
    """
    
    def __init__(self, curve=None, generator=None):
        """
        Инициализация - выбор кривой, образующего элемента
        Возможно конструирование по готовой кривой и образующему
        """
        
        #
        # Для простоты буду использовать кривую из ГОСТ 34.10 
        #
        
        self._curve = gost3410.CURVES['id-tc26-gost-3410-2012-512-paramSetA'] if curve is None else curve
        self._generator = (self._curve.x, self._curve.y) if generator is None else generator
        self._modulus = self._curve.p
        
        trace('[EllipticCurveDiffieHellmanProcessor]', 
              f'Using group of order {self._modulus} over curve {self._curve.name}')
        
    
    def generate_private_key(self):
        """
        Генерирование закрытого ключа
        """
        
        return random.randint(int(2), int(self.order - 2))
    
    
    def get_public_key(self, private_key):
        """
        Получение открытого ключа по закрытому
        """
        
        return self.exponentiate(private_key, self._generator)
    
    
    def exponentiate(self, scalar, element):
        """
        Умножение точки на скаляр
        """
        
        if not self._curve.contains(element):
            return None
        
        return multiply_by_scalar(self._curve, scalar, element)
    
    
    def element_to_bytes(self, element):
        """
        Конвертация элемента группы в байтовое представление
        """
        
        return gost3410.pub_marshal(element)
    
    
    @property
    def group(self):
        """
        Получение группы
        """
        
        return self._curve
    
    
    @property
    def generator(self):
        """
        Получение образующего элемента
        """
        
        return self._generator
    
    
    @property
    def order(self):
        """
        Получение порядка группы
        """
        
        return self._modulus

----

In [12]:
cipher_suites = ['ECDHE_KUZNYECHIK_MGM_STREEBOG', 'DHE_AES_GCM_SHA256']

In [13]:
def resolve_cipher_suite(cipher_suite):
    """
    Получение примитивов по строковому идентификатору
    """
        
    #
    # Пока захардкодил это все
    #
        
    if cipher_suite == 'ECDHE_KUZNYECHIK_MGM_STREEBOG':
        return EllipticCurveDiffieHellmanProcessor, KuznyechikMGM, Streebog
        
    elif cipher_suite == 'DHE_AES_GCM_SHA256':
        return DiffieHellmanProcessor, AESGCM, SHA256
        
    raise ValueError(f'Unsupported cipher suite {cipher_suite}')

----

In [14]:
def hmac(key, *args, hasher=Streebog):
    """
    Вычисление HMAC (согласно Р 50.1.113 - 2016)
    Информационная технология
    КРИПТОГРАФИЧЕСКАЯ ЗАЩИТА ИНФОРМАЦИИ
    Криптографические алгоритмы, сопутствующие применению алгоритмов 
    электронной цифровой подписи и функции хэширования
    """
    
    def xor(lhs, rhs):
        return bytes([l ^^ r for l, r in zip(lhs, rhs)])
    
    IPAD = b'\x36' * 64
    OPAD = b'\x5c' * 64
    
    K_star = key + (b'\x00' * (512 - len(key)))
    
    inner_hash = hash_digest(hasher, xor(K_star, IPAD), *args)
    return hash_digest(hasher, xor(K_star, OPAD), inner_hash)

In [15]:
def test_hmac():
    """
    Р 50.1.113 - 2016
    Информационная технология
    КРИПТОГРАФИЧЕСКАЯ ЗАЩИТА ИНФОРМАЦИИ
    Криптографические алгоритмы, сопутствующие применению алгоритмов 
    электронной цифровой подписи и функции хэширования
    
    4.1.1 HMAC_GOSTR3411_2012_256
    
    Приложение А. Контрольные примеры
    """
    
    K = b'\x00\x01\x02\x03\x04\x05\x06\x07\x08\x09\x0a\x0b\x0c\x0d\x0e\x0f' + \
        b'\x10\x11\x12\x13\x14\x15\x16\x17\x18\x19\x1a\x1b\x1c\x1d\x1e\x1f'
    
    T = b'\x01\x26\xbd\xb8\x78\x00\xaf\x21\x43\x41\x45\x65\x63\x78\x01\x00'
    
    expected = b'\xa1\xaa\x5f\x7d\xe4\x02\xd7\xb3\xd3\x23\xf2\x99\x1c\x8d\x45\x34' + \
        b'\x01\x31\x37\x01\x0a\x83\x75\x4f\xd0\xaf\x6d\x7c\xd4\x92\x2e\xd9'
    
    assert hmac(K, T, hasher=Streebog) == expected

In [16]:
#
# Протестирую HMAC на контрольном примере из рекомендаций по стандартизации
#

test_hmac()

----

In [17]:
class GOST3410Signer(object):
    """
    Объект-создатель подписей по ГОСТ 34.10 - 2012
    """
    
    def __init__(self):
        """
        Инициализация кривой и ключевой пары
        """
        
        self._curve = gost3410.CURVES['id-tc26-gost-3410-2012-512-paramSetA']
        self._private_key = gost3410.prv_unmarshal(get_random_bytes(64))
        self._public_key = gost3410.public_key(self._curve, self._private_key)
        
        
    def sign(self, *args):
        """
        Подпись произвольного набора байтовых объектов
        """
        
        digest = hash_digest(Streebog, *args)
        return gost3410.sign(self._curve, self._private_key, digest)
        
        
    @property
    def public_key(self):
        """
        Получение открытого ключа
        """
        
        return self._public_key
    
    
    @property
    def private_key(self):
        """
        Получение закрытого ключа
        """
        
        return self._private_key

    
    @property
    def curve(self):
        """
        Получение используемой кривой
        """
        
        return self._curve
    
    
    @property
    def generator(self):
        """
        Получение образующего группы точек кривой
        """
        
        return self._curve.x, self._curve.y

In [18]:
def verify_signature(curve, public_key, signature, *args):
    """
    Проверка подписи по ГОСТ 34.10 - 2012
    """
    
    digest = hash_digest(Streebog, *args)
    return gost3410.verify(curve, public_key, digest, signature)

----

## Реализация участников протокола

In [19]:
Offer = namedtuple('Offer', ['cipher_suite', 'group', 'generator'])

In [20]:
class GenericParticipant(object):
    """
    Класс-абстракция над участником протокола, который получает сертификат на свой открытый
    ключ и может осуществлять подпись сообщений
    """
    
    def __init__(self, ca, *, fake_data=None, demo_cert_error=False):
        """
        Инициализация - получение идентификатора, генерирование ключевой пары, 
        получение сертификата
        """
        
        self._id = uuid.uuid4()
        self._signer = GOST3410Signer()
        
        if fake_data is not None:
            #
            # Демонстрация неудачного выполнения протокола
            #
            
            if demo_cert_error:
                #
                # Тут демонстрация неудачи получения серификата
                #
                
                self._signer._public_key = fake_data
                self._certificate = ca.issue_certificate(self)
            else:
                #
                # Тут как бы сертификат и открытый ключ правильные, а закрытый ключ противник не угадал
                #
                
                self._certificate = ca.issue_certificate(self)
                self._signer._public_key = fake_data
        else:
            #
            # Стандартное выполнение
            #
            
            self._certificate = ca.issue_certificate(self)
        
        self._n = None
        
        trace('[GenericParticipant]', f'Certificate = {self._certificate}')
        
        
    def sign(self, *args):
        """
        Подписывание произвольных данных
        """
        
        return self._signer.sign(*args)
        
        
    @property
    def public_key(self):
        """
        Получение открытого ключа подписи
        """
        
        return self._signer.public_key
    
    
    @property
    def identifier(self):
        """
        Получение идентификатора
        """
        
        return self._id
    
    
    @property
    def certificate(self):
        """
        Получение сертификата
        """
        
        return self._certificate
    
    
    @property
    def curve(self):
        """
        Получение кривой, используемой для подписи
        """
        
        return self._signer.curve
    
    
    def _get_point_for_schnorr_proof(self):
        #
        # Костыль, но все же работает
        #
        
        self._n = gost3410.prv_unmarshal(get_random_bytes(64))
        return multiply_by_scalar(self._signer.curve, 
                                  self._n, self._signer.generator)
    
    
    def _get_response(self, chellenge):
        return self._n + chellenge * self._signer.private_key

In [21]:
class TLS13Participant(GenericParticipant):
    """
    Обобщенный участник общения по TLS 1.3
    """
    
    def __init__(self, ca, *, fake_data=None, demo_cert_error=False):
        """
        Инициализация - по факту просто сохраняю ссылку на СА
        """
        
        super(TLS13Participant, self).__init__(ca, 
                                               fake_data=fake_data, 
                                               demo_cert_error=demo_cert_error)
        self._ca = ca
    
    
    def check_certificate(self, identifier: uuid.UUID, public_key, cert: Certificate):
        """
        Проверка полученного сертификата
        """
        
        if self._ca.is_revoked(cert):
            trace('[TLS13Participant]', f'Revoked certificate')
            return False
            
        id_as_bytes = str(identifier).encode()
        pk_as_bytes = gost3410.pub_marshal(public_key)
        
        if not verify_signature(self._ca.curve, 
                                self._ca.public_key, 
                                cert.signature, 
                                id_as_bytes, 
                                pk_as_bytes):
            trace('[TLS13Participant]', f'Invalid certificate')
            return False
        
        return True
        
        
    @staticmethod
    def check_mac(key, mac, *args):
        """
        Проверка имитовставки
        """
        
        return mac == hmac(key, *args)
    
    
    @staticmethod
    def check_signature(curve, public_key, signature, *args):
        """
        Проверка подписи
        """
        
        return verify_signature(curve, public_key, signature, *args)

In [22]:
class TLS13Client(TLS13Participant):
    """
    Клиент для TLS 1.3
    """
        
    S2C = 0
    C2S = 1
    
    class AuthMode(Enum):
        """
        Режим аутентицикации
        """
        
        OWA = 1  # Односторонняя
        TWA = 2  # Двусторонняя
    
    
    def __init__(self, ca, cipher_suite, *, fake_data=None, demo_cert_error=False):
        """
        Инициализация - разбор набора примитивов
        """
        
        super(TLS13Client, self).__init__(ca, fake_data=fake_data, 
                                          demo_cert_error=demo_cert_error)
        
        self._key_exchange_type, self._cipher_type, self._hasher = resolve_cipher_suite(cipher_suite)
        self._cipher_suite = cipher_suite
        self._session_keys = {}
        
        trace('[TLS13Client]', f'Using cipher suite {cipher_suite}')
        
        
    def establish_connection(self, server, auth_mode: AuthMode):
        """
        Установление соединения с сервером
        """
        
        key_exchange = self._key_exchange_type()
        
        ephemeral_dh_sk = key_exchange.generate_private_key()
        ephemeral_dh_pk = key_exchange.get_public_key(ephemeral_dh_sk)
        client_nonce    = get_random_bytes(64)
        
        trace('[TLS13Client]', f'Ephemeral DH public key: {ephemeral_dh_pk}')
        trace('[TLS13Client]', f'Nonce: {hexlify(client_nonce)}')
        
        offer = Offer(self._cipher_suite, 
                      key_exchange.group, 
                      key_exchange.generator)
        
        #
        # А теперь запрос на сервер
        #
        
        is_one_way = auth_mode == TLS13Client.AuthMode.OWA
        server_hello = server.get_hello(self.identifier, ephemeral_dh_pk, client_nonce, 
                                        offer, is_one_way)
        
        if server_hello is None:
            trace('[TLS13Client]', f'Server refused the connection')
            return False
        
        #
        # Распакую ответ
        #
        
        ephemeral_dh_server_pk, server_nonce, cipher_suite, c1, c2, c3, c4 = server_hello
        
        if cipher_suite != self._cipher_suite:
            trace('[TLS13Client]', f'Unsupported cipher suite {cipher_suite}')
            return False
        
        #
        # Общий секрет DH
        #
        
        ephemeral_dh_common = key_exchange.exponentiate(ephemeral_dh_sk, ephemeral_dh_server_pk)
        if ephemeral_dh_common is None:
            trace(f'[TLS13Client]', 'Server ECDH public key is not on curve')
        
        #
        # Вырабатываю ключи для симметричного шифрования и имитовставки
        #
        
        client_packet = b''.join([key_exchange.element_to_bytes(ephemeral_dh_pk),
                                  client_nonce, str(offer).encode()])
        
        server_packet = b''.join([key_exchange.element_to_bytes(ephemeral_dh_server_pk),
                                  server_nonce, cipher_suite.encode()])
        
        key_derivation_data = b''.join([key_exchange.element_to_bytes(ephemeral_dh_common),
                                        client_packet, server_packet])
        
        encryption_key, mac_key = HKDF(key_derivation_data, self._cipher_type.KEY_SIZE, 
                                       b'\x00' * 16, self._hasher, 2)
        
        #
        # Расшифровываю полученные пакеты
        #
        
        cipher = self._cipher_type(encryption_key)
        
        try:
            cert_requested     = cipher.decrypt(*c1)
            server_certificate = cipher.decrypt(*c2)
            server_signature   = cipher.decrypt(*c3)
            server_mac         = cipher.decrypt(*c4)
        except ValueError:
            trace('[TLS13Client]', f'Malformed server response')
            return False
        
        #
        # Проверяю полученнные данные
        #
        
        if is_one_way == cert_requested:
            #
            # Запросили сертификат в ходе односторонней аутентификации или наоборот
            #
            
            trace('[TLS13Client]', f'Invalid client certificate request')
            return False
        
        if not self.check_certificate(server.identifier, server.public_key, 
                                      pickle.loads(server_certificate)):
            trace('[TLS13Client]', f'Invalid server certificate')
            return False
        
        if not TLS13Participant.check_signature(server.curve, server.public_key, server_signature, 
                                                client_packet, server_packet, c1[1], c2[1]):
            trace('[TLS13Client]', f'Malformed server response')
            return False
        
        if not TLS13Participant.check_mac(mac_key, server_mac, 
                                          client_packet, server_packet, c1[1], c2[1], c3[1]):
            trace('[TLS13Client]', f'Malformed server response')
            return False
            
        #
        # Если требуется двусторонняя аутентификация, то зашлю на сервер сертификат
        #
        
        if not is_one_way:
            c5 = cipher.encrypt(pickle.dumps(self.certificate))
            
            signature = self.sign(client_packet, server_packet, c1[1], c2[1], c3[1], c4[1], c5[1])
            c6 = cipher.encrypt(signature)
            
            mac = hmac(mac_key, client_packet, server_packet, c1[1], c2[1], c3[1], c4[1], c5[1], c6[1])
            c7 = cipher.encrypt(mac)
            
            if not server.process_client_certificate(self, c5, c6, c7):
                trace('[TLS13Client]', f'Server refused the connection')
                return False
        
        #
        # Теперь я генерирую ключи шифрования
        #
        
        key_derivation_data = b''.join([key_exchange.element_to_bytes(ephemeral_dh_common),
                                        client_packet, server_packet, c1[1], c2[1], c3[1], c4[1]])
            
        self._session_keys[server.identifier] = HKDF(key_derivation_data, self._cipher_type.KEY_SIZE,
                                                     b'\x00' * 16, self._hasher, 2)
        
        trace('[TLS13Client]', f'Server accepted the connection')
        
        trace('[TLS13Client]', 
              f's -> c key: {hexlify(self._session_keys[server.identifier][TLS13Client.S2C])}')
        trace('[TLS13Client]', 
              f'c -> s key: {hexlify(self._session_keys[server.identifier][TLS13Client.C2S])}')
        
        return True
    
    
    def send_wait_receive_message(self, server, message: str):
        """
        Отправка сообщения на сервер и получение ответа
        """
        
        if server.identifier not in self._session_keys:
            trace('[TLS13Client]', f'Connection with server {server.identifier} is not established yet')
            return
        
        cipher = self._cipher_type(self._session_keys[server.identifier][TLS13Client.C2S])
        
        response = server._receive_message_and_respond(self, cipher.encrypt(message.encode()))
        if response is None:
            trace('[TLS13Client]', f'Malformed server response')
        
        try:
            cipher = self._cipher_type(self._session_keys[server.identifier][TLS13Client.S2C])
            return cipher.decrypt(*response).decode()
        except ValueError:
            trace('[TLS13Client]', f'Malformed server response')
            
            
    def change_keys(self, server):
        """
        Смена ключей по инициативе клиента
        """
        
        if server.identifier not in self._session_keys:
            trace('[TLS13Client]', f'Connection with server {server.identifier} is not established yet')
            return
        
        #
        # Меняю у себя
        #
        
        self._change_keys_internal(server.identifier)
        
        #
        # Меняю на сервере
        #
        
        server._change_keys_internal(self.identifier)
        
        
    def _change_keys_internal(self, server_id):
        s2c_key = HKDF(self._session_keys[server_id][TLS13Client.S2C], self._cipher_type.KEY_SIZE, 
                       b'\x00' * 16, self._hasher, 1)
        
        c2s_key = HKDF(self._session_keys[server_id][TLS13Client.C2S], self._cipher_type.KEY_SIZE, 
                       b'\x00' * 16, self._hasher, 1)
        
        trace('[TLS13Client]', f'New s -> c key: {hexlify(s2c_key)}')
        trace('[TLS13Client]', f'New c -> s key: {hexlify(c2s_key)}')
        
        self._session_keys[server_id] = (s2c_key, c2s_key)

In [23]:
class TLS13Server(TLS13Participant):
    """
    Сервер для TLS 1.3
    """
    
    CIPHER_TYPE            = 0
    HASHER                 = 1
    HANDSHAKE_ENC_KEY      = 2
    HANDSHAKE_MAC_KEY      = 3
    HANDSHAKE_VALUES       = 4
    HANDSHAKE_KEY_EXCHANGE = 5
    
    S2C = 0
    C2S = 1
    
    
    def __init__(self, ca, cipher_suites):
        """
        Инициализация - сохранение настроек
        """
        
        super(TLS13Server, self).__init__(ca)
        
        self._cipher_suites = cipher_suites
        self._session_keys  = {}
        self._mapped_states = {}
        
        
    def get_hello(self, client_id, ephemeral_dh_client_pk, client_nonce, offer, is_one_way):
        """
        Получение ServerHello
        """
        
        #
        # Провверяю, что поддерживается вообще набор примитивов
        #
        
        cipher_suite, group, generator = offer
        
        if cipher_suite not in self._cipher_suites:
            trace('[TLS13Server]', f'Unsupported cipher suite {cipher_suite}')
            return None
        
        trace('[TLS13Server]', f'Using cipher suite {cipher_suite} with client {client_id}')
        
        #
        # Получаю примитивы
        #
        
        key_exchange_type, cipher_type, hasher = resolve_cipher_suite(cipher_suite)
        key_exchange = key_exchange_type(group, generator)
        
        #
        # Вырабатываю ключи DH и nonce, после чего получаю общий секрет DH
        #
        
        ephemeral_dh_sk = key_exchange.generate_private_key()
        ephemeral_dh_pk = key_exchange.get_public_key(ephemeral_dh_sk)
        server_nonce    = get_random_bytes(64)
        
        trace('[TLS13Server]', f'Ephemeral DH public key: {ephemeral_dh_pk}')
        trace('[TLS13Server]', f'Nonce: {hexlify(client_nonce)}')
        
        ephemeral_dh_common = key_exchange.exponentiate(ephemeral_dh_sk, ephemeral_dh_client_pk)
        if ephemeral_dh_common is None:
            trace(f'[TLS13Client]', 'Client ECDH public key is not on curve')
        
        #
        # Вырабатываю ключи для симметричного шифрования и имитовставки
        #
        
        client_packet = b''.join([key_exchange.element_to_bytes(ephemeral_dh_client_pk),
                                  client_nonce, str(offer).encode()])
        
        server_packet = b''.join([key_exchange.element_to_bytes(ephemeral_dh_pk),
                                  server_nonce, cipher_suite.encode()])
        
        key_derivation_data = b''.join([key_exchange.element_to_bytes(ephemeral_dh_common),
                                        client_packet, server_packet])
        
        encryption_key, mac_key = HKDF(key_derivation_data, cipher_type.KEY_SIZE, 
                                       b'\x00' * 16, hasher, 2)
        
        #
        # Рассчитываю нужные значения
        # 
        
        cipher = cipher_type(encryption_key)
        
        cert_request = bytes([not is_one_way])
        c1 = cipher.encrypt(cert_request)
        
        cert = pickle.dumps(self.certificate)
        c2 = cipher.encrypt(cert)
        
        signature = self.sign(client_packet, server_packet, c1[1], c2[1])
        c3 = cipher.encrypt(signature)
        
        mac = hmac(mac_key, client_packet, server_packet, c1[1], c2[1], c3[1])
        c4 = cipher.encrypt(mac)
        
        #
        # Тут будут сохранены нужные значения
        #
        
        self._mapped_states[client_id] = [None, None, None, None, None, None]
        
        #
        # Теперь выработаю ключи (если это односторонняя аутентификация)
        #
        
        if is_one_way:
            self._session_keys[client_id] = TLS13Server._generate_keys(key_exchange, cipher_type.KEY_SIZE, 
                                                                       hasher, ephemeral_dh_common, 
                                                                       client_packet, server_packet, 
                                                                       c1, c2, c3, c4)
        
        #
        # Сохраняю остальное и возвращаю клиенту
        #
        
        self._mapped_states[client_id][TLS13Server.CIPHER_TYPE]       = cipher_type
        self._mapped_states[client_id][TLS13Server.HASHER]            = hasher
        self._mapped_states[client_id][TLS13Server.HANDSHAKE_ENC_KEY] = encryption_key
        self._mapped_states[client_id][TLS13Server.HANDSHAKE_MAC_KEY] = mac_key
        self._mapped_states[client_id][TLS13Server.HANDSHAKE_VALUES]  = (ephemeral_dh_common, client_packet, 
                                                                         server_packet, c1, c2, c3, c4)
        self._mapped_states[client_id][TLS13Server.HANDSHAKE_KEY_EXCHANGE] = key_exchange
        
        return ephemeral_dh_pk, server_nonce, cipher_suite, c1, c2, c3, c4
    
    
    def process_client_certificate(self, client, c5, c6, c7):
        """
        Проверка сертификата клиента
        """
        
        client_id = client.identifier
        
        #
        # Достану ранее сохраненные параметры
        #
        
        cipher_type    = self._mapped_states[client_id][TLS13Server.CIPHER_TYPE]
        hasher         = self._mapped_states[client_id][TLS13Server.HASHER]
        encryption_key = self._mapped_states[client_id][TLS13Server.HANDSHAKE_ENC_KEY]
        mac_key        = self._mapped_states[client_id][TLS13Server.HANDSHAKE_MAC_KEY]
        
        ephemeral_dh_common, client_packet, server_packet, c1, c2, c3, c4 = \
            self._mapped_states[client_id][TLS13Server.HANDSHAKE_VALUES]
        
        key_exchange   = self._mapped_states[client_id][TLS13Server.HANDSHAKE_KEY_EXCHANGE]
        
        #
        # Расшифрую полученный пакет
        #
        
        cipher = cipher_type(encryption_key)
        
        try:
            client_certificate = cipher.decrypt(*c5)
            client_signature   = cipher.decrypt(*c6)
            client_mac         = cipher.decrypt(*c7)
        except ValueError:
            trace('[TLS13Server]', f'Malformed client response')
            return False
        
        #
        # проверю все полученные данные
        #
        
        if not self.check_certificate(client.identifier, client.public_key, 
                                      pickle.loads(client_certificate)):
            trace('[TLS13Server]', f'Invalid client certificate')
            return False
        
        if not TLS13Participant.check_signature(client.curve, client.public_key, client_signature, 
                                                client_packet, server_packet, 
                                                c1[1], c2[1], c3[1], c4[1], c5[1]):
            trace('[TLS13Server]', f'Malformed client response')
            return False
        
        if not TLS13Participant.check_mac(mac_key, client_mac, client_packet, server_packet, 
                                          c1[1], c2[1], c3[1], c4[1], c5[1], c6[1]):
            trace('[TLS13Server]', f'Malformed client response')
            return False
        
        #
        # Все чудесно - генерирую ключи
        #
        
        key_derivation_data = b''.join([key_exchange.element_to_bytes(ephemeral_dh_common),
                                        client_packet, server_packet, c1[1], c2[1], c3[1], c4[1]])
            
        self._session_keys[client_id] = TLS13Server._generate_keys(key_exchange, cipher_type.KEY_SIZE, 
                                                                   hasher, ephemeral_dh_common, client_packet, 
                                                                   server_packet, c1, c2, c3, c4)
        
        return True
            
            
    def change_keys(self, client):
        """
        Смена ключей по инициативе сервера
        """
        
        if client.identifier not in self._session_keys:
            trace('[TLS13Client]', f'Connection with client {client.identifier} is not established yet')
            return
        
        #
        # Меняю у себя
        #
        
        self._change_keys_internal(client.identifier)
        
        #
        # Меняю на сервере
        #
        
        client._change_keys_internal(self.identifier)
        
        
    def _change_keys_internal(self, client_id):
        s2c_key = HKDF(self._session_keys[client_id][TLS13Client.S2C], 
                       self._mapped_states[client_id][TLS13Server.CIPHER_TYPE].KEY_SIZE, 
                       b'\x00' * 16, self._mapped_states[client_id][TLS13Server.HASHER], 1)
        
        c2s_key = HKDF(self._session_keys[client_id][TLS13Client.C2S], 
                       self._mapped_states[client_id][TLS13Server.CIPHER_TYPE].KEY_SIZE, 
                       b'\x00' * 16, self._mapped_states[client_id][TLS13Server.HASHER], 1)
        
        trace('[TLS13Client]', f'New s -> c key: {hexlify(s2c_key)}')
        trace('[TLS13Client]', f'New c -> s key: {hexlify(c2s_key)}')
        
        self._session_keys[client_id] = (s2c_key, c2s_key)
    
    
    def _receive_message_and_respond(self, client, encrypted_message):
        client_id = client.identifier
        
        try:
            cipher = self._mapped_states[client_id][TLS13Server.CIPHER_TYPE](
                self._session_keys[client_id][TLS13Server.C2S])
            
            message = cipher.decrypt(*encrypted_message).decode()
            print(f'Received message "{message}"')
            
            cipher = self._mapped_states[client_id][TLS13Server.CIPHER_TYPE](
                self._session_keys[client_id][TLS13Server.S2C])
            
            return cipher.encrypt(f'Hello, {message}!'.encode())
        except ValueError:
            trace('[TLS13Server]', f'Malformed client message')
    
    
    @staticmethod
    def _generate_keys(key_exchange, key_size, hasher, ephemeral_dh_common, 
                       client_packet, server_packet, c1, c2, c3, c4):
        key_derivation_data = b''.join([key_exchange.element_to_bytes(ephemeral_dh_common),
                                        client_packet, server_packet, c1[1], c2[1], c3[1], c4[1]])
            
        s2c_key, c2s_key = HKDF(key_derivation_data, key_size, 
                                b'\x00' * 16, hasher, 2)
        
        trace('[TLS13Server]', f's -> c key: {hexlify(s2c_key)}')
        trace('[TLS13Server]', f'c -> s key: {hexlify(c2s_key)}')
        
        return s2c_key, c2s_key

In [24]:
class TLS13CertificationAuthority(object):
    """
    Удостоверющий центр для TLS 1.3
    """
    
    def __init__(self):
        """
        Инициализация - генерирование ключевой пары
        """
        
        self._signer = GOST3410Signer()
        
        self._certificates = set()
        self._revoked_certificates = set()
        
        trace('[TLS13CertificationAuthority]', 
              f'Public key = {self._signer.public_key}')
        
        
    @property
    def public_key(self):
        """
        Получение открытого ключа
        """
        
        return self._signer.public_key
    
    
    @property
    def curve(self):
        """
        Получение кривой, используемой для подписи
        """
        
        return self._signer.curve
        
        
    def issue_certificate(self, participant: GenericParticipant):
        """
        Выдача сертификата на открытый ключ
        """
        
        #
        # Проверю, что клиент действительно владеет закрытым ключом
        #
        
        if not self._check_participant(participant):
            trace('[TLS13CertificationAuthority]', 
                  f'Participant cannot prove a valid private key possession')
            
            return None
        
        #
        # Все чудесно, подписываю ключ и отдаю сертификат
        #
        
        id_as_bytes = str(participant.identifier).encode()
        pk_as_bytes = gost3410.pub_marshal(participant.public_key)
        
        cert = Certificate(participant.identifier, participant.public_key, 
                           self._signer.sign(id_as_bytes, pk_as_bytes))
        
        self._certificates.add(cert)
        
        trace('[TLS13CertificationAuthority]', 
              f'Issued certificate for {participant.identifier}. Total certificates: {len(self._certificates)}')
        
        return cert
    
    
    def revoke_certificate(self, participant: GenericParticipant):
        """
        Отзыв сертификата
        """
        
        #
        # А можно ли вообще отзывать сертификат?
        #
        
        if not self._check_participant(participant):
            trace('[TLS13CertificationAuthority]', 
                  f'Participant cannot prove a valid private key possession')
            
            return 
        
        if participant.certificate not in self._certificates:
            #
            # А отзывать то и нечего
            #
            
            return
            
        self._revoked_certificates.add(participant.certificate)
        self._certificates.discard(participant.certificate)
        
        trace('[TLS13CertificationAuthority]', 
              f'List of revoked certificates now has {len(self._revoked_certificates)} elements')
        
        
    def is_revoked(self, cert: Certificate):
        """
        Проверка на факт отзыва сертификата
        """
        
        return cert in self._revoked_certificates
    
    
    def _check_participant(self, participant: GenericParticipant) -> bool:
        Q = participant._get_point_for_schnorr_proof()
        c = gost3410.prv_unmarshal(get_random_bytes(64))
        t = participant._get_response(c)
        
        tP  = multiply_by_scalar(self._signer.curve, t, self._signer.generator)
        cPK = multiply_by_scalar(self._signer.curve, c, participant.public_key)
        
        return tP == add_points(self._signer.curve, Q, cPK)

----
## Демонстрация удачного выполнения протоколов

In [25]:
# Создание удостоверяющего центра
ca = TLS13CertificationAuthority()

[TRACE] [TLS13CertificationAuthority] Public key = (12122282932933569293641183943791646023095002098321829876539511163241259319018622984196822367718855324768845468516680812427753433159172594674350880235694869, 11971790682455570631981692498412110022677454817314544077180194131513908419142635689714821861357273605079966974100035545496454456130714854243184137593550525)


In [26]:
# Создание сервера
srv = TLS13Server(ca, cipher_suites)

[TRACE] [TLS13CertificationAuthority] Issued certificate for fccda92f-e47a-446d-8bb9-e7db100f9560. Total certificates: 1
[TRACE] [GenericParticipant] Certificate = Certificate(id=UUID('fccda92f-e47a-446d-8bb9-e7db100f9560'), public_key=(6850994505451387564911768305568509478384073776712394543195498780102783735650295464299269673417510195564302899962460753544736164791362877953309449806096045, 5559194221738753263374554251289604640520282784850186380539652834930366390889955781043828387659503487023738241000730209494652216039797363299483297133443231), signature=b'F){5\xbd\x170R>\xf7\xd4\xeb\x18#\x0b\xc3\xad\x0ei\x80B\xfe\xb8\xcd\x1b_\xca\xdc4r\x82\xceWr\xcb!S\xb3;/\xcf\xbbnz6\xac\x02\xad\x11,\x80\xaa\xae.\xed\x9b\xcdY\xa5\xf2k7\xf62\xc5\x80\xa0\x99n\xe4\xc6\x96u|\xd8\x15s6 \xc2\x80q\xf8:\x97K\xb5\xd7\xe9\xa1\x82p\xd2FR\x13\x13\x07g\x04\xcc\xe83\xaddl\x1e\xb5\x82\xd5+\xdeZ\x813\xb1x\\\x1e\xb5\x1c\xb8]\xf2\x85\xac\x18\x1c')


In [27]:
# Первый валидный пользователь (использует ECDH, Кузнечик в режиме MGM и Стрибог)
alice = TLS13Client(ca, cipher_suite=cipher_suites[0])

[TRACE] [TLS13CertificationAuthority] Issued certificate for 57431f80-bd0e-45d3-ac90-87581bd915c1. Total certificates: 2
[TRACE] [GenericParticipant] Certificate = Certificate(id=UUID('57431f80-bd0e-45d3-ac90-87581bd915c1'), public_key=(1677781238484222883450496353799492563845651382739838485404522815707747210379769102161499560103914861972101928032578273854032551425689074952860828085426123, 5304163492884426681911243922009334682911855157169046056107919973922525884475904340981388913167658500463202629851002715367998277155216552982097550811017247), signature=b'\xd1\x0f\x8a\xcf\xe6\x9f\xcc\xd6\xcb\xf6-\x0c\xda\x979\xbc\xbc\xb3\xa0\xb1\xb1C\xd4\xb3\xc1L\x80\xdaw\xee\x9f\xce\x1d9>\xfe\x8f\xe6K\x8e\xfbY\xc4\x06\x87\xd9\xe9\x90\xc6\x11"\xe6\x15i84\x9c<~B3\xac\xb1?\xcd\x9b\x16,q\x80\xdau\xbb\xdd\xfaR\xacZ\xf4f]\x9f\x85S\x8a\xda\xb8\xe7\xf53\x01\xe6\xbe:!\x82n\xc9\x97\x8a\xf1\x1f\x13\xb7.@\x9b\x0e\xb9\x1dJ!\xb7[G)\x1f\x11\x9a\x9bk\xc7h\x94\xd3M\x96\xbc')
[TRACE] [TLS13Client] Using cipher suite EC

In [28]:
# Второй валидный пользователь (использует DH, AES в режиме GCM и SHA256)
bob = TLS13Client(ca, cipher_suite=cipher_suites[1])

[TRACE] [TLS13CertificationAuthority] Issued certificate for 5e62ed4a-0c2e-476b-aaae-3e993df50dd6. Total certificates: 3
[TRACE] [GenericParticipant] Certificate = Certificate(id=UUID('5e62ed4a-0c2e-476b-aaae-3e993df50dd6'), public_key=(5137572185352682447987272854729904016890456218988825489735171509098699826220745681821923759119405068088941620553716210691937610082771334594225476463621888, 12426200203615076894729791594077278829235921533944362598998367051944051879441780194689202679346194051940292682378021340092149439544770524143099147631524657), signature=b"\xdc]I\x9a\xb6\xcd,o\x91\rIqUF\xda\x8a\xa0\xa735e\xa7\xb1\xfb\xbe\x11\x85\xf3mn\xf4\x9d;\xc6U\xfb\xe5\xb63m\xf1u\x99\xc9\x86\x05\xab\x86\\\xbf\x9d3\xce'tF\xaa\x8b\xd5\xa3\x0ek3\xd6\xceE\x89\xbd\xbdG\xadF\xdb\xe9\xacl^\x84P\xbe\x1f\x8f\x07\x9c\x11N\x93\xc8?\x1f\xdeY3\xe8\x94&&\xba\x9e[Q(\xc9\xa6c\xff\xee\xda\xec=\xed88\xcb\xb7\xdf$@\xa3\x93\xc7ZeI#\x10\xd6v")
[TRACE] [TLS13Client] Using cipher suite DHE_AES_GCM_SHA256


In [29]:
# Устанавливаю соединение с односторонней аутентификацией
alice.establish_connection(srv, TLS13Client.AuthMode.OWA)

[TRACE] [EllipticCurveDiffieHellmanProcessor] Using group of order 13407807929942597099574024998205846127479365820592393377723561443721764030073546976801874298166903427690031858186486050853753882811946569946433649006083527 over curve id-tc26-gost-3410-2012-512-paramSetA
[TRACE] [TLS13Client] Ephemeral DH public key: (3064891939127257748779378112161644246317133644937086013527859754514842960160389959214178352262296229108526421300102577097915004596814295972945686335521333, 4684239468546385713762487044498250014546155942467788387439697236792020419702522932538584735293813235366773700598850546448527116866115885831320033503349385)
[TRACE] [TLS13Client] Nonce: b'1d9c3b851a643d71ce26c268fe97e825873c97ccb53cad51fa27bc4dd3af1e02e49fdc22f3af582e3279d4c0fa08013b768acf489e5a8c66018987fab8c9ecc9'
[TRACE] [TLS13Server] Using cipher suite ECDHE_KUZNYECHIK_MGM_STREEBOG with client 57431f80-bd0e-45d3-ac90-87581bd915c1
[TRACE] [EllipticCurveDiffieHellmanProcessor] Using group of order 134078079299425970995

True

In [30]:
# Устанавливаю соединение с двусторонней аутентификацией
bob.establish_connection(srv, TLS13Client.AuthMode.TWA)

[TRACE] [DiffieHellmanProcessor] Using group of order 10127393884953880848248842158634879058630734809852619481809061340370250061713889001050993091287651562668861927501368128495872426698402770564501153806430819
[TRACE] [TLS13Client] Ephemeral DH public key: 2868766898966701854972361820996183265365068998529622844121970366673300381735213679128263815087033617544652418036390345535053870046684774177997018122568871
[TRACE] [TLS13Client] Nonce: b'910e2b781de569c6d584f833eb3a7202b12e2f37bdbb189205491a92c075f9fada966d747f9db8cbadfb11056d89f76d2b03d929c847cbd03a0b42a0041a35ea'
[TRACE] [TLS13Server] Using cipher suite DHE_AES_GCM_SHA256 with client 5e62ed4a-0c2e-476b-aaae-3e993df50dd6
[TRACE] [DiffieHellmanProcessor] Using group of order 10127393884953880848248842158634879058630734809852619481809061340370250061713889001050993091287651562668861927501368128495872426698402770564501153806430819
[TRACE] [TLS13Server] Ephemeral DH public key: 2691990242724646380190004356354289117152976366115641653745892

True

In [31]:
# Отправлю сообщение
alice.send_wait_receive_message(srv, 'world')

Received message "world"


'Hello, world!'

In [32]:
# Сменю ключи по инициативе клиента
alice.change_keys(srv)

[TRACE] [TLS13Client] New s -> c key: b'0927763af8fa046c13b833e03195a62eb70884387ad8c60560ec2a46badd24a5'
[TRACE] [TLS13Client] New c -> s key: b'e5fecbaa1c1a7c7cc3a6543041fe1b6dd90b323fe573ab3676a1c8d501d825a6'
[TRACE] [TLS13Client] New s -> c key: b'0927763af8fa046c13b833e03195a62eb70884387ad8c60560ec2a46badd24a5'
[TRACE] [TLS13Client] New c -> s key: b'e5fecbaa1c1a7c7cc3a6543041fe1b6dd90b323fe573ab3676a1c8d501d825a6'


In [33]:
# И еще раз отправлю сообщение, чтобы было видно, что ключи успешно поменялись
alice.send_wait_receive_message(srv, 'world')

Received message "world"


'Hello, world!'

In [34]:
# Отправлю сообщение другим клиентом
bob.send_wait_receive_message(srv, 'crypto')

Received message "crypto"


'Hello, crypto!'

In [35]:
# Смена ключей по инициативе сервера
srv.change_keys(bob)

[TRACE] [TLS13Client] New s -> c key: b'cb7e9b8b721193d40bec112668791695'
[TRACE] [TLS13Client] New c -> s key: b'f5271d988743c6156b93c808307f5f2b'
[TRACE] [TLS13Client] New s -> c key: b'cb7e9b8b721193d40bec112668791695'
[TRACE] [TLS13Client] New c -> s key: b'f5271d988743c6156b93c808307f5f2b'


In [36]:
# Все по-прежнему успешно отправляется
bob.send_wait_receive_message(srv, 'crypto')

Received message "crypto"


'Hello, crypto!'

----
## Отзыв сертификата

In [37]:
# Третий валидный пользователь
carol = TLS13Client(ca, cipher_suite=cipher_suites[0])

[TRACE] [TLS13CertificationAuthority] Issued certificate for 183eeb3e-4f8f-45fa-889d-6cc5d24f490b. Total certificates: 4
[TRACE] [GenericParticipant] Certificate = Certificate(id=UUID('183eeb3e-4f8f-45fa-889d-6cc5d24f490b'), public_key=(5304251142124592499029057546810863421469230281411341638422545508645730085582198978563356351246306791132722996980395781613434523019202672085394914579976841, 3798505913410763457278880414350784662627829782425698715396958221013618783589505180292035761234364130679625579439314226148524854878658318718860776225695058), signature=b">\x027\x96>M\x02\xde\xbc\xce\xc4?>\x0bo\\\xcf\xfe\xaa\xfc`\xd9\xdf\x8a\xba\x81\xbe2<\xb8\x00V\x9c\xc4H\xfd\xa2\x95\xd8\x89\x8b(u\n\xecn\xc69&c\xb9\x00\xdf2\xab\xd6\xb4e\xcf1\xc9\x15V\xfem\xb4\x1c-A\rP\xd1\x0fo\x01\xad\xf0\xd9u,\x0e'\xd9Q\xc8\xdb\xd5U\x99\xcc<\xf2\x07bw\xb8v`\xf9\x8a\x9eZ\xda\xb2\xe2\x83<\xd6\r\xc0s\xda\r>X\xe1'\xbda\r\xb0\x0c\xbc\x8bH\x02\xfc`")
[TRACE] [TLS13Client] Using cipher suite ECDHE_KUZNYECHIK_MGM_STREEBOG


In [38]:
# Отзову сертификат Кэрол
ca.revoke_certificate(carol)

[TRACE] [TLS13CertificationAuthority] List of revoked certificates now has 1 elements


In [39]:
# Попробую подключиться, используя двустороннюю аутентификацию
carol.establish_connection(srv, TLS13Client.AuthMode.TWA)

[TRACE] [EllipticCurveDiffieHellmanProcessor] Using group of order 13407807929942597099574024998205846127479365820592393377723561443721764030073546976801874298166903427690031858186486050853753882811946569946433649006083527 over curve id-tc26-gost-3410-2012-512-paramSetA
[TRACE] [TLS13Client] Ephemeral DH public key: (9269145656548327907210525779246347644461348265025165842924144267823632453180042162377670738299523021248012124986308402689490186303624432494825489736552517, 5123489364057525540004458958297528818755190874749084234579561911755088448100868465331908922311096203384694578462285780574491273703751765908947741582223220)
[TRACE] [TLS13Client] Nonce: b'a171009a63532ec22e647df2341d4b5fd1943f738639610bd77f84782416b39a3589b14593ec5e2f37db6358722dd86faf990951fdde348584b72883f3bc35a0'
[TRACE] [TLS13Server] Using cipher suite ECDHE_KUZNYECHIK_MGM_STREEBOG with client 183eeb3e-4f8f-45fa-889d-6cc5d24f490b
[TRACE] [EllipticCurveDiffieHellmanProcessor] Using group of order 134078079299425970995

False

----
## Демонстрация неудачного выполнения протоколов

In [40]:
# Тут Мэллори пытается установить подключение с двусторонней аутентификацией,
# используя сертификат не на свой ключ (я просто подменю тут открытый ключ для простоты)
mallory = TLS13Client(ca, cipher_suite=cipher_suites[0], 
                      fake_data=bob.public_key)

[TRACE] [TLS13CertificationAuthority] Issued certificate for d72bf42f-83ea-4120-9a1e-c77c74853d36. Total certificates: 4
[TRACE] [GenericParticipant] Certificate = Certificate(id=UUID('d72bf42f-83ea-4120-9a1e-c77c74853d36'), public_key=(9441143587655706848931793392276151996072617831927053904976161883745363161779898190949308089578382375433148325170782367298628454803001963587531227354814531, 1156229304918075376735037674476089193517367359594476328946534320406588509115566218564596510601598727315445389777706112253974727210846524626084881846702663), signature=b'\x15\xe7\xce\xa7\xbcIf;\r\xa4\x83c|e\xeanu:\xf67\xd7\xa0\x83L\x84M3)\x90J\xbf\xde\xd9\x8b\x96\x18\x8bj\xc3\xcb\xf9\x1e\x07\xc2bl\xe4\x14\x1e|\x9d\x97\xc88\xa3\xd1f\x87~\t\xbb\x85h\xcb\xe1\x8c\x14\xf1\x8c\x9c;\x1c\xac\xa7\xbd\xec\xc2\xf6\xc9\xee$\xf5\x91\xe4\xf7w\xeb\xe3\x10\x86qD\xe0\xa6\xef\xb7\xac^~&\xc7\xfc\xba\x16~\xac\x80ga\xf9\x0f\xb0=\x9fX^\xaf$\xacW\xf9\x0c\xab\xe1\xd7q\xd5\xbd')
[TRACE] [TLS13Client] Using cipher suite ECDHE_

In [41]:
mallory.establish_connection(srv, TLS13Client.AuthMode.TWA)

[TRACE] [EllipticCurveDiffieHellmanProcessor] Using group of order 13407807929942597099574024998205846127479365820592393377723561443721764030073546976801874298166903427690031858186486050853753882811946569946433649006083527 over curve id-tc26-gost-3410-2012-512-paramSetA
[TRACE] [TLS13Client] Ephemeral DH public key: (381674843511948559666855393701321130360633815211210071848075228051107755919219273579099911816330532273119460141609992175366883428856418940812071500288877, 10990934295216331037463945270508380694954458786914786642062871641854296845293438156473764577951162656565454009176191499858173160744044692097363961873282912)
[TRACE] [TLS13Client] Nonce: b'f471777c42152f5ee57be362614450ff4f4a83dc7ae01db0032099216919741cf4193946e5bd44dd5ae055d8bb8fbb8a9398989a7d52ced152af6bebaa4aeea9'
[TRACE] [TLS13Server] Using cipher suite ECDHE_KUZNYECHIK_MGM_STREEBOG with client d72bf42f-83ea-4120-9a1e-c77c74853d36
[TRACE] [EllipticCurveDiffieHellmanProcessor] Using group of order 134078079299425970995

False

In [42]:
# Мэллори теперь пытается получить сертификат на ключ Боба, не зная его закрытого ключа
mallory = TLS13Client(ca, cipher_suite=cipher_suites[0], 
                      fake_data=bob.public_key, 
                      demo_cert_error=True)

[TRACE] [TLS13CertificationAuthority] Participant cannot prove a valid private key possession
[TRACE] [GenericParticipant] Certificate = None
[TRACE] [TLS13Client] Using cipher suite ECDHE_KUZNYECHIK_MGM_STREEBOG
